In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import folium
from folium.plugins import TimestampedGeoJson
from math import radians, sin, cos, sqrt, atan2
from branca.colormap import linear

In [2]:
data = pd.read_csv("hourly_df_semaine2229.csv")
df_real = pd.DataFrame(data)
df_real['taux_dispo'] =  (df_real['Available'] + df_real['Offline']/3)/3
df_real = df_real[['timestamp','latitude','longitude','taux_occup','taux_dispo']]



In [3]:
# Définir les limites de la grille (Paris)
lat_min, lat_max = 48.81, 48.92
lon_min, lon_max = 2.255, 2.42
step = 0.005  # Taille des carrés (grille de 0.005°)

# Créer les carrés de la grille
latitudes = np.arange(lat_min, lat_max, step)
longitudes = np.arange(lon_min, lon_max, step)


In [ ]:



# Liste pour stocker les données des carrés
grille_data = []

for i in range(len(latitudes) - 1):
    for j in range(len(longitudes) - 1):
        # Calculer le centre du carré
        center_lat = (latitudes[i] + latitudes[i + 1]) / 2
        center_lon = (longitudes[j] + longitudes[j + 1]) / 2
        center_point = Point(center_lon, center_lat)

        # Définir les limites du carré (polygon)
        bounds = [
            (latitudes[i], longitudes[j]),
            (latitudes[i], longitudes[j + 1]),
            (latitudes[i + 1], longitudes[j + 1]),
            (latitudes[i + 1], longitudes[j]),
        ]
        polygon = Polygon(bounds)

        # Vérifier si le centre est dans Paris (tous arrondissements combinés)
        if not arrondissements_gdf.geometry.unary_union.contains(center_point):
            continue  # Ignorer les carrés hors des arrondissements de Paris

        # Vérifier si le centre est proche des zones exclues (fonction personnalisée)
        if is_near_exclusion(center_lat, center_lon):  # Fonction à définir
            continue

        # Calculer la distance à la station la plus proche
        distance_to_station = stations_distance(center_lat, center_lon)  # Fonction à définir

        # Ajouter les données du carré à la liste
        grille_data.append({
            "id": f"{i}_{j}",
            "center_lat": center_lat,
            "center_lon": center_lon,
            "polygon": polygon,
            "distance_to_station": distance_to_station,
        })

# Convertir les données des carrés en DataFrame
grille_df = pd.DataFrame(grille_data)

# Normaliser la distance pour une visualisation (facultatif)
grille_df["normalized_distance"] = grille_df["distance_to_station"] / grille_df["distance_to_station"].max()

# Sauvegarder la grille en CSV (optionnel)
grille_df.to_csv("grille_carre_paris.csv", index=False)

# Visualisation de la grille avec des couleurs en fonction des distances
import folium

m2 = folium.Map(location=[(lat_min + lat_max) / 2, (lon_min + lon_max) / 2], zoom_start=12)

for _, row in grille_df.iterrows():
    folium.Polygon(
        locations=[(lat, lon) for lat, lon in row["polygon"].exterior.coords],
        color=None,
        fill=True,
        fill_color=distance_to_color(row["distance_to_station"], grille_df["distance_to_station"].max()),  # Fonction personnalisée
        fill_opacity=0.3
    ).add_to(m2)

# Afficher la carte
m2
